##### Copyright 2018 The TensorFlow Authors.

# Neural style transfer

## Setup


In [ ]:
import os
import tensorflow as tf
# Loada dei modelli compressi da tf
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

import requests

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools
 


class Hyperparameters():
  def __init__(self):
    self.optimizer = 'ADAM'
    self.lr = 0.001
    self.momentum = 0.9
    self.BETA1 = 0.9
    self.BETA2 = 0.9
    self.initial_accumulator_value = 0.01
    self.epsilon = 1e-7
    self.rho = 0.95
    self.epochs = 100
    self.steps_per_epoch = 100
    self.style_weight = 100
    self.content_weight = 1e-1
    self.variational_weight = 0
  def reset(self):
    self.optimizer = 'ADAM'
    self.lr = 0.001
    self.momentum = 0.9
    self.BETA1 = 0.9
    self.BETA2 = 0.9
    self.initial_accumulator_value = 0.01
    self.epsilon = 1e-7
    self.rho = 0.95
    self.epochs = 100
    self.steps_per_epoch = 100
    self.style_weight = 100
    self.content_weight = 1e-1
    self.variational_weight = 0


# classe con utils functions
class Utils():

  #trasforma un tensore in un oggetto immagine (Pil.Image)
  def tensor_to_image(self,tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
      assert tensor.shape[0] == 1
      tensor = tensor[0]
    return PIL.Image.fromarray(tensor)
  # ritorna un oggetto immagine caricato da un path datogli, setta la massima dimensione a 512
  def load_img(self,path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img
  # printa a schermo l'immagine datagli
  def imshow(self,image, title=None):
    if len(image.shape) > 3:
      image = tf.squeeze(image, axis=0)

    
    if title:
      plt.title(title)
  def clip_0_1(self,image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)



from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')







class ModelStructure(tf.keras.models.Model):
  def __init__(self,util):
    super(ModelStructure, self).__init__()
    # questi parametri sono settati di default con l'inizializzazione del costruttore
    # possono essere modificati duranti il training
    self.util = util
    url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
    response = requests.get(url)
    file = open("Tuebingen_Neckarfront.jpg", "wb")
    file.write(response.content)
    file.close()
    self.content_path = "Tuebingen_Neckarfront.jpg"
    self.style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')
    self.training_image_path= "Tuebingen_Neckarfront.jpg"
    self.vgg = None
    self.vgg =  self.vgg_layers(['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1','block4_conv2','block5_conv1','block5_conv2'])
    self.style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
    self.content_layers = ['block4_conv2']
    self.num_style_layers = len(self.style_layers)
    self.num_content_layers = len(self.content_layers)
    self.opt = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    self.content_image = tf.Variable(util.load_img(self.content_path))
    self.style_image = tf.Variable(util.load_img(self.style_path))
    self.image = self.content_image
    self.style_weight=1e-2
    self.content_weight=1e4
    self.total_variation_weight=50
    

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [self.gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}
  
  def printVggLayers(self):
    for layer in self.content_layers:
      print(layer)
    for layer1 in self.style_layers:
      print(layer1)

  def set_content_image(self,image_path):
    self.content_path = image_path
    self.content_image = tf.Variable(self.util.load_img(image_path))
  def set_style_image(self,image_path):
    self.style_path = image_path
    self.style_image = tf.Variable(self.util.load_img(image_path))
  def set_training_image(self,image_path):
    self.training_image_path=image_path
    #training_image_path="/content/gdrive/MyDrive/rumore_bianco.jpg"
    #img = tf.io.read_file(training_image_path)
    self.image = tf.Variable(self.util.load_img(image_path))
    
    
  #ritorna un modello rispetto agli output dati in layer names
  def vgg_layers(self,layer_names):
    self.vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet', pooling='avg')# carica vgg senza i fully connected layers finali (vgg usata per la classification)
    self.vgg.trainable = False
    outputs = [self.vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([self.vgg.input], outputs)
    return model
  
  def set_loss_weights(self,style_weight, content_weight, variation_weight):
    self.style_weight = style_weight
    self.content_weight = content_weight
    self.total_variation_weight=variation_weight

  def set_vgg_layers(self,style_layers, content_layers):
    self.num_style_layers = len(style_layers)
    self.num_content_layers = len(content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.vgg = self.vgg_layers(style_layers+content_layers)
  
  def set_optimizer(self,optimizer,learning_rate,momentum,beta_1, beta_2, initial_accumulator_value, epsilon, rho):
    if optimizer == 'ADAM':
      self.opt = tf.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2 = beta_2, epsilon=epsilon)
    elif optimizer == 'ADAGRAD': 
      self.opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate, initial_accumulator_value=initial_accumulator_value,epsilon=epsilon)
    elif optimizer == 'RMSPROP':
      self.opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate,rho=rho, momentum=momentum, epsilon=epsilon)
    elif optimizer == 'ADADELTA':
      self.opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate, rho=rho, epsilon=epsilon)
    elif optimizer == 'ADAMAX':
      self.opt = tf.keras.optimizers.Adamax(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)

  # computa la gram matrix per lo style error
  def gram_matrix(self,input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)
  
  def style_content_loss(self,outputs, style_targets, content_targets):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean(tf.square(style_outputs[name]-style_targets[name])) 
                           for name in style_outputs.keys()])
    style_loss *= self.style_weight / self.num_style_layers

    content_loss = tf.add_n([tf.reduce_mean(tf.square(content_outputs[name]-content_targets[name])) 
                             for name in content_outputs.keys()])
    content_loss *= self.content_weight / self.num_content_layers
    loss = style_loss + content_loss
   
    return loss 
  def reset(self):
    url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
    response = requests.get(url)
    file = open("Tuebingen_Neckarfront.jpg", "wb")
    file.write(response.content)
    file.close()
    self.content_path = "Tuebingen_Neckarfront.jpg"
    self.style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')
    self.training_image_path= "Tuebingen_Neckarfront.jpg"
    self.vgg = None
    self.vgg =  self.vgg_layers(['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1','block4_conv2','block5_conv1','block5_conv2'])
    self.style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
    self.content_layers = ['block4_conv2']
    self.num_style_layers = len(self.style_layers)
    self.num_content_layers = len(self.content_layers)
    self.opt = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    self.content_image = tf.Variable(util.load_img(self.content_path))
    self.style_image = tf.Variable(util.load_img(self.style_path))
    self.image = self.content_image
    self.style_weight=1e-2
    self.content_weight=1e4
    self.total_variation_weight=50
@tf.function()
def train_step(model,image, style, content):
  with tf.GradientTape() as tape:
    outputs = model(image)
    loss = model.style_content_loss(outputs,style,content)
    loss += model.total_variation_weight*tf.image.total_variation(image)
    
  grad = tape.gradient(loss, image)
  model.opt.apply_gradients([(grad, image)])
  image.assign(util.clip_0_1(image))
  return loss



def model_training_and_output(epochs, steps_per_epoch, model):
  step = 0
  image = tf.Variable(model.util.load_img(model.training_image_path))
  model.set_content_image(model.content_path)
  model.set_style_image(model.style_path)
  style = model(model.style_image)['style']
  content = model(model.content_image)['content']
  losses=[]
  step=0
  for n in range(epochs):
    for m in range(steps_per_epoch):
      
      
      loss = train_step(model,image, style,content)
      losses=np.append(losses, loss[0])
      step += 1
      print(".", end='')
    #display.clear_output(wait=True)
    print(loss)
    display.display(model.util.tensor_to_image(image))
    print("Train step: {}".format(step))
  lo1=losses[200:1499]  
  plt.plot(lo1)
  plt.title('TOTAL LOSS OVER FIRST 15 EPOCHS (without first 2)')
  plt.savefig('losses1.png')
  file_name1 = 'losses1.png'
  files.download(file_name1)
  plt.show() 
  lo2=losses[1500:]
  plt.plot(lo2)
  plt.title('TOTAL LOSS OVER EPOCHS > 15')
  plt.show() 
  plt.savefig('losses2.png')
  file_name2 = 'losses2.png'
  files.download(file_name2)
  return image





Mounted at /content/gdrive


# MODEL INITIALIZATION AND HYPERPARAMETERS INITIALIZATION

In [ ]:
util = Utils()
model = ModelStructure(util)
param = Hyperparameters()

param.reset()
model.set_optimizer(param.optimizer,param.lr,param.momentum,param.BETA1,param.BETA2,param.initial_accumulator_value,param.epsilon,param.rho)
model.set_loss_weights(param.style_weight,param.content_weight,param.variational_weight)

# WHITE NOISE IMAGE CREATION

In [ ]:
training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
model.util.tensor_to_image(training_image).save("noise.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)

# VARIE TRY

# Relazione. Subsection 4.1

# 4.1.1

In [ ]:
model = ModelStructure(util)
param.reset()
model.set_optimizer(param.optimizer,param.lr,param.momentum,param.BETA1,param.BETA2,param.initial_accumulator_value,param.epsilon,param.rho)
model.set_loss_weights(param.style_weight,param.content_weight,param.variational_weight)
image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_minotaur.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:

#different content/style weight (1000) and total wariation weight=100
model = ModelStructure(util)
param.reset()
model.set_optimizer(param.optimizer,param.lr,param.momentum,param.BETA1,param.BETA2,param.initial_accumulator_value,param.epsilon,param.rho)
param.style_weight = 5e-3
param.content_weight = 5000
param.variational_weight = 50
model.set_loss_weights(param.style_weight,param.content_weight,param.variational_weight)


  # posso cambiare le immagini
  url = 'https://www.artble.com/imgs/c/d/1/98090/the_shipwreck_of_the_minotaur.jpg'
  response = requests.get(url)
  file = open("the_shipwreck_of_the_minotaur.jpg", "wb")
  file.write(response.content)
  file.close()
  model.set_style_image("the_shipwreck_of_the_minotaur.jpg")
  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)
  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1', 'block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_minotaur(fixed).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.1.2

In [ ]:
# 4.1.2

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.005
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 100
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 100
  content_weight = 1e-1
  variational_weight = 0
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1200px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'
  
  response = requests.get(url)

  file = open("starry_night.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_style_image("starry_night.jpg")
  #model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  #content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_starry.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.001
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 30
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 1e-3
  content_weight = 3000
  variational_weight = 0
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1200px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'
  
  response = requests.get(url)

  file = open("starry_night.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_style_image("starry_night.jpg")
  #model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  #content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_starry(fixed).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.1.3

In [ ]:
# 4.1.3

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.001
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 100
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 100
  content_weight = 1e-1
  variational_weight = 0
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  url = 'https://www.theartpostblog.com/wp-content/uploads/2018/04/img-urlo-much.jpg'
  response = requests.get(url)

  file = open("scream.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_style_image("scream.jpg")
 
  #c'è anche per il content image
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_scream.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.1.3 (our Parameters)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.005
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 100
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 1e-1
  content_weight = 100
  variational_weight = 500
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  url = 'https://www.theartpostblog.com/wp-content/uploads/2018/04/img-urlo-much.jpg'
  response = requests.get(url)

  file = open("scream.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_style_image("scream.jpg")
  #model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_scream(fixed).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.1.4

In [ ]:

# 4.1.4

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.005
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 50
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 100
  content_weight = 1e-2
  variational_weight = 0
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  url = 'https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg'
  response = requests.get(url)

  file = open("femme_nue.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_style_image("femme_nue.jpg")
  #model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_femme.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.1.4

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.005
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 30
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 1e-2
  content_weight = 1000
  variational_weight = 0
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  url = 'https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg'
  response = requests.get(url)

  file = open("femme_nue.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_style_image("femme_nue.jpg")
  #model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_femme(fixed).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.1.5

In [ ]:

# 4.1.5

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.005
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 50
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 100
  content_weight = 1e-2
  variational_weight = 0
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig2_kandinsky.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:



# 4.1.5

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.005
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 60
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 1e-1
  content_weight = 5000
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  
  

  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  
  response = requests.get(url)

  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  model.util.tensor_to_image(training_image).save("noise.jpg")
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig2_kandinsky(fixed).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# Relazione. Subsection 4.2

# 4.2.1

In [ ]:
# 4.2.1


util = Utils()
model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.005
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 30
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-3
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini  
model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image

model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
model.util.tensor_to_image(training_image).save("noise.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)

  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(1,1).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.2

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.005
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 20
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-2
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(1,2).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.3

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.005
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 20
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-1
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)

model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(1,3).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.4

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.005
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 20
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 10000
content_weight = 100
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)

model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(1,4).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.2.2


In [ ]:
# 4.2.5

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.005
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 20
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-3
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(2,1).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.6

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 10
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-2
print("alpha - beta ratio: ")
print(content_weight/style_weight)
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block1_conv2']
style_layers = ['block1_conv1', 'block2_conv2']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(2,2).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.7

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-1
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(2,3).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.8

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 30
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 1000
content_weight = 10
print("alpha - beta ratio: ")
print(content_weight / style_weight)

variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
model.util.tensor_to_image(training_image).save("noise.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block1_conv1']
style_layers = ['block1_conv1', 'block2_conv2']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(2,4).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.2.3

In [ ]:
# 4.2.9

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-3
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(3,1).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.10

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-2
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(3,2).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.11

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-1
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(3,3).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.12

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-1
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(3,4).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.2.4

In [ ]:
# 4.2.13

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-3
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(4,1).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.14

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-2
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(4,2).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.15

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-1
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(4,3).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.16

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 1000
content_weight = 10
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(4,4).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# 4.2.5

In [ ]:
# 4.2.17

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-3
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1', 'block5_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(5,1).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.18

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-2
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1', 'block5_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(5,2).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.19

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 100
content_weight = 1e-1
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1', 'block5_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(5,3).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# 4.2.20

util = Utils()
model = ModelStructure(util)
 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM'
lr = 0.001
momentum = 0.9
BETA1 = 0.9
BETA2 = 0.9
initial_accumulator_value = 0.01
epsilon = 1e-7
rho = 0.95
epochs = 150
steps_per_epoch = 100
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  

  # per cambiare i pesi delle loss posso fare
style_weight = 1000
content_weight = 10
print("alpha - beta ratio: ")
print(int(content_weight / style_weight))
variational_weight = 0
model.set_loss_weights(style_weight,content_weight,variational_weight)


model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
model.set_content_image("Tuebingen_Neckarfront.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)
  #posso anche cambiare i layers
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv2', 'block3_conv1', 'block4_conv1', 'block5_conv1']
model.set_vgg_layers(style_layers, content_layers)
  
image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = 'paper_fig3(5,4).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# Customized training

# Content Image = Training Image

In [ ]:
# 4.3.1

util = Utils()
model = ModelStructure(util) 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM' #@param["ADAM", "ADAMAX" , "ADAGRAD", "ADADELTA", "ADAGRAD", "SGD", "RMSPROP"]
lr = 0.005 #@param {type:"number"}
momentum = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
BETA1 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
BETA2 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
initial_accumulator_value = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
epsilon = 1e-7 
rho = 0.95 
epochs = 10 #@param {type:"number"}
steps_per_epoch = 20 #@param {type:"slider", min:10, max:1000, step:10}
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)

  # per cambiare i pesi delle loss posso fare
style_weight = 1e3 #@param {type:"number"}
content_weight = 1e-4 #@param {type:"number"}
variational_weight = 100 #@param {type:"number"}
model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
style_url = 'https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg' #@param {type:"raw"}
response = requests.get(style_url)
file = open("femme_nue.jpg", "wb")
file.write(response.content)
file.close()
model.set_style_image("femme_nue.jpg")
#c'è anche per il content image
content_url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg' #@param {type:"raw"}
response = requests.get(content_url)
file = open("Tuebingen_Neckarfront.jpg", "wb")
file.write(response.content)
file.close()
model.set_content_image("Tuebingen_Neckarfront.jpg")

  #training image
model.set_training_image("Tuebingen_Neckarfront.jpg")

  #posso anche cambiare i layers
content_layers = ['block5_conv2'] #@param {type:"raw"}
style_layers = ['block2_conv1', 'block3_conv1'] #@param {type:"raw"}
model.set_vgg_layers(style_layers, content_layers)
  
  #training procedure
image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'Tuebigen_Femme.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# Max vs Average Pooling

In [ ]:
import os
import tensorflow as tf
# Loada dei modelli compressi da tf
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

import requests

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools
 





# classe con utils functions
class Utils():

  #trasforma un tensore in un oggetto immagine (Pil.Image)
  def tensor_to_image(self,tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
      assert tensor.shape[0] == 1
      tensor = tensor[0]
    return PIL.Image.fromarray(tensor)
  # ritorna un oggetto immagine caricato da un path datogli, setta la massima dimensione a 512
  def load_img(self,path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img
  # printa a schermo l'immagine datagli
  def imshow(self,image, title=None):
    if len(image.shape) > 3:
      image = tf.squeeze(image, axis=0)

    
    if title:
      plt.title(title)
  def clip_0_1(self,image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)



from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')







class ModelStructure(tf.keras.models.Model):
  def __init__(self,util):
    super(ModelStructure, self).__init__()
    # questi parametri sono settati di default con l'inizializzazione del costruttore
    # possono essere modificati duranti il training
    self.util = util
    url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
    response = requests.get(url)
    file = open("Tuebingen_Neckarfront.jpg", "wb")
    file.write(response.content)
    file.close()
    self.content_path = "Tuebingen_Neckarfront.jpg"
    self.style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')
    self.training_image_path= "Tuebingen_Neckarfront.jpg"
    self.vgg = None
    self.pool = 'avg'
    self.vgg =  self.vgg_layers(['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1','block4_conv2','block5_conv1','block5_conv2'], self.pool)
    self.style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
    self.content_layers = ['block4_conv2']
    self.num_style_layers = len(self.style_layers)
    self.num_content_layers = len(self.content_layers)
    self.opt = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    self.content_image = tf.Variable(util.load_img(self.content_path))
    self.style_image = tf.Variable(util.load_img(self.style_path))
    self.image = self.content_image
    self.style_weight=1e-2
    self.content_weight=1e4
    self.total_variation_weight=50
    

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [self.gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}
  
  def printVggLayers(self):
    for layer in self.content_layers:
      print(layer)
    for layer1 in self.style_layers:
      print(layer1)

  def set_content_image(self,image_path):
    self.content_path = image_path
    self.content_image = tf.Variable(self.util.load_img(image_path))
  def set_style_image(self,image_path):
    self.style_path = image_path
    self.style_image = tf.Variable(self.util.load_img(image_path))
  def set_training_image(self,image_path):
    self.training_image_path=image_path
    #training_image_path="/content/gdrive/MyDrive/rumore_bianco.jpg"
    #img = tf.io.read_file(training_image_path)
    self.image = tf.Variable(self.util.load_img(image_path))
    
    
  #ritorna un modello rispetto agli output dati in layer names
  def vgg_layers(self,layer_names, pool):
    if pool=='max':
      self.vgg =  tf.keras.applications.VGG19(include_top=False, weights='imagenet') #max pooling is default in vgg19
    elif pool=='avg':
      self.vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet', pooling= pool)# carica vgg senza i fully connected layers finali (vgg usata per la classification)
    self.vgg.trainable = False
    outputs = [self.vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([self.vgg.input], outputs)
    return model
  
  def set_loss_weights(self,style_weight, content_weight, variation_weight):
    self.style_weight = style_weight
    self.content_weight = content_weight
    self.total_variation_weight=variation_weight

  def set_vgg_layers(self,style_layers, content_layers, pool):
    self.num_style_layers = len(style_layers)
    self.num_content_layers = len(content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.vgg = self.vgg_layers(style_layers+content_layers, pool)
  
  def set_optimizer(self,optimizer,learning_rate,momentum,beta_1, beta_2, initial_accumulator_value, epsilon, rho):
    if optimizer == 'ADAM':
      self.opt = tf.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2 = beta_2, epsilon=epsilon)
    elif optimizer == 'ADAGRAD': 
      self.opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate, initial_accumulator_value=initial_accumulator_value,epsilon=epsilon)
    elif optimizer == 'RMSPROP':
      self.opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate,rho=rho, momentum=momentum, epsilon=epsilon)
    elif optimizer == 'ADADELTA':
      self.opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate, rho=rho, epsilon=epsilon)
    elif optimizer == 'ADAMAX':
      self.opt = tf.keras.optimizers.Adamax(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)

  # computa la gram matrix per lo style error
  def gram_matrix(self,input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)
  
  def style_content_loss(self,outputs, style_targets, content_targets):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean(tf.square(style_outputs[name]-style_targets[name])) 
                           for name in style_outputs.keys()])
    style_loss *= self.style_weight / self.num_style_layers

    content_loss = tf.add_n([tf.reduce_mean(tf.square(content_outputs[name]-content_targets[name])) 
                             for name in content_outputs.keys()])
    content_loss *= self.content_weight / self.num_content_layers
    loss = style_loss + content_loss
   
    return loss 




@tf.function()
def train_step(model,image, style, content):
  with tf.GradientTape() as tape:
    outputs = model(image)
    loss = model.style_content_loss(outputs,style,content)
    loss += model.total_variation_weight*tf.image.total_variation(image)
    
  grad = tape.gradient(loss, image)
  model.opt.apply_gradients([(grad, image)])
  image.assign(util.clip_0_1(image))
  return loss



def model_training_and_output(epochs, steps_per_epoch, model):
  step = 0
  image = tf.Variable(model.util.load_img(model.training_image_path))
  model.set_content_image(model.content_path)
  model.set_style_image(model.style_path)
  style = model(model.style_image)['style']
  content = model(model.content_image)['content']
  losses=[]
  step=0
  for n in range(epochs):
    for m in range(steps_per_epoch):
      
      
      loss = train_step(model,image, style,content)
      losses=np.append(losses, loss[0])
      step += 1
      print(".", end='')
    #display.clear_output(wait=True)
    print(loss)
    display.display(model.util.tensor_to_image(image))
    print("Train step: {}".format(step))
  lo1=losses[200:1499]  
  plt.plot(lo1)
  plt.title('TOTAL LOSS OVER FIRST 15 EPOCHS (without first 2)')
  plt.savefig('losses1.png')
  file_name1 = 'losses1.png'
  files.download(file_name1)
  plt.show() 
  lo2=losses[1500:]
  plt.plot(lo2)
  plt.title('TOTAL LOSS OVER EPOCHS > 15')
  plt.show() 
  plt.savefig('losses2.png')
  file_name2 = 'losses2.png'
  files.download(file_name2)
  return image



####FINE SETUP








# 4.3.1

util = Utils()
model = ModelStructure(util) 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM' #@param["ADAM", "ADAMAX" , "ADAGRAD", "ADADELTA", "ADAGRAD", "SGD", "RMSPROP"]
lr = 0.005 #@param {type:"number"}
momentum = 0.29 #@param {type:"slider", min:0, max:1, step:0.01}
BETA1 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
BETA2 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
initial_accumulator_value = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
epsilon = 1e-7 
rho = 0.95 
epochs = 50 #@param {type:"number"}
steps_per_epoch = 100 #@param {type:"slider", min:10, max:1000, step:10}
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)

  # per cambiare i pesi delle loss posso fare
style_weight = 1e-2 #@param {type:"number"}
content_weight = 1000 #@param {type:"number"}
variational_weight = 0 #@param {type:"number"}
model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
style_url = 'https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg' #@param {type:"raw"}
response = requests.get(style_url)
file = open("femme_nue.jpg", "wb")
file.write(response.content)
file.close()
model.set_style_image("femme_nue.jpg")
  #c'è anche per il content image
content_url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg' #@param {type:"raw"}
response = requests.get(content_url)
file = open("Tuebingen_Neckarfront.jpg", "wb")
file.write(response.content)
file.close()
model.set_content_image("Tuebingen_Neckarfront.jpg")
  #training image
training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
model.util.tensor_to_image(training_image).save("noise.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)

  #posso anche cambiare i layers
content_layers = ['block5_conv2'] #@param {type:"raw"}
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'] #@param {type:"raw"}
pooling='avg' #@param["avg" , "max"]
model.set_vgg_layers(style_layers, content_layers, pooling)
  
  #training procedure
image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '4.3(AVG).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# Different Content Layers

In [ ]:
# 4.3.1

util = Utils()
model = ModelStructure(util) 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM' #@param["ADAM", "ADAMAX" , "ADAGRAD", "ADADELTA", "ADAGRAD", "SGD", "RMSPROP"]
lr = 0.005 #@param {type:"number"}
momentum = 0.29 #@param {type:"slider", min:0, max:1, step:0.01}
BETA1 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
BETA2 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
initial_accumulator_value = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
epsilon = 1e-7 
rho = 0.95 
epochs = 30 #@param {type:"number"}
steps_per_epoch = 100 #@param {type:"slider", min:10, max:1000, step:10}
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)

  # per cambiare i pesi delle loss posso fare
style_weight = 1e-2 #@param {type:"number"}
content_weight = 1000 #@param {type:"number"}
variational_weight = 0 #@param {type:"number"}
model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
style_url = 'https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg' #@param {type:"raw"}
response = requests.get(style_url)
file = open("femme_nue.jpg", "wb")
file.write(response.content)
file.close()
model.set_style_image("femme_nue.jpg")
  #c'è anche per il content image
content_url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg' #@param {type:"raw"}
response = requests.get(content_url)
file = open("Tuebingen_Neckarfront.jpg", "wb")
file.write(response.content)
file.close()
model.set_content_image("Tuebingen_Neckarfront.jpg")
  #training image
training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
model.util.tensor_to_image(training_image).save("noise.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)

  #posso anche cambiare i layers
content_layers = ['block4_conv2', 'block5_conv2'] #@param {type:"raw"}
#content_layers = ['block3_conv2', 'block4_conv2', 'block5_conv2']
#content_layers = ['block3_conv2']
#content_layers = ['block1_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'] #@param {type:"raw"}
model.set_vgg_layers(style_layers, content_layers)
  
  #training procedure
image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'Tuebigen_Femme.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# Different Optimizers

In [ ]:
# 4.3.1
util = Utils()
model = ModelStructure(util) 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAMAX' #@param["ADAM", "ADAMAX", "ADADELTA", "ADAGRAD", "SGD", "RMSPROP"]
#optimizer = 'ADAGRAD'
#optimizer = 'ADADELTA'  
#optimizer = 'ADAGRAD' 
#optimizer = 'RMSPROP' 
#optimizer = 'SGD'
lr = 0.005 #@param {type:"number"}
momentum = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
BETA1 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
BETA2 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
initial_accumulator_value = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
epsilon = 1e-7 
rho = 0.95 
epochs = 30 #@param {type:"number"}
steps_per_epoch = 100 #@param {type:"slider", min:10, max:1000, step:10}
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)

  # per cambiare i pesi delle loss posso fare
style_weight = 1e-2 #@param {type:"number"}
content_weight = 1000 #@param {type:"number"}
variational_weight = 0 #@param {type:"number"}
model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
style_url = 'https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg' #@param {type:"raw"}
response = requests.get(style_url)
file = open("femme_nue.jpg", "wb")
file.write(response.content)
file.close()
model.set_style_image("femme_nue.jpg")
  #c'è anche per il content image
content_url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg' #@param {type:"raw"}
response = requests.get(content_url)
file = open("Tuebingen_Neckarfront.jpg", "wb")
file.write(response.content)
file.close()
model.set_content_image("Tuebingen_Neckarfront.jpg")
  #training image
training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
model.util.tensor_to_image(training_image).save("noise.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)

  #posso anche cambiare i layers
content_layers = ['block5_conv2'] #@param {type:"raw"}
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'] #@param {type:"raw"}
model.set_vgg_layers(style_layers, content_layers)
  
  #training procedure
image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'Tuebigen_Femme.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# Different momentum, betas, accumulator...

In [ ]:
# 4.3.1

util = Utils()
model = ModelStructure(util) 

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
optimizer = 'ADAM' #@param["ADAM", "ADAMAX" , "ADAGRAD", "ADADELTA", "ADAGRAD", "SGD", "RMSPROP"]
lr = 0.005 #@param {type:"number"}
momentum = 0.29 #@param {type:"slider", min:0, max:1, step:0.01}
BETA1 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
BETA2 = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
initial_accumulator_value = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
epsilon = 1e-7 
rho = 0.95 
epochs = 30 #@param {type:"number"}
steps_per_epoch = 100 #@param {type:"slider", min:10, max:1000, step:10}
  
model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)

  # per cambiare i pesi delle loss posso fare
style_weight = 1e-2 #@param {type:"number"}
content_weight = 1000 #@param {type:"number"}
variational_weight = 0 #@param {type:"number"}
model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
style_url = 'https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg' #@param {type:"raw"}
response = requests.get(style_url)
file = open("femme_nue.jpg", "wb")
file.write(response.content)
file.close()
model.set_style_image("femme_nue.jpg")
  #c'è anche per il content image
content_url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg' #@param {type:"raw"}
response = requests.get(content_url)
file = open("Tuebingen_Neckarfront.jpg", "wb")
file.write(response.content)
file.close()
model.set_content_image("Tuebingen_Neckarfront.jpg")
  #training image
training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
model.util.tensor_to_image(training_image).save("noise.jpg")
training_image_path="noise.jpg"
model.set_training_image(training_image_path)

  #posso anche cambiare i layers
content_layers = ['block5_conv2'] #@param {type:"raw"}
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'] #@param {type:"raw"}
model.set_vgg_layers(style_layers, content_layers)
  
  #training procedure
image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'Tuebigen_Femme.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 10 
  content_weight = 10
  variational_weight = 10
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image

  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  



  image=model_training_and_output(epochs,steps_per_epoch,model)

file_name = '(10,10,10), ADAM(lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# same parameters of the paper

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 0
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  #c'è anche per il content image
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
  
  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(200000,2,50), ADAM (lr=0.01), style all 5, block4_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# learning rate greather (0.1)



if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAGRAD'
  lr = 0.1
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.1
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
 

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(200000,2,50), ADAM (lr=0.1), style all 5, block_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# learning rate lower (0.001)


if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.001
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
 

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(200000,2,50), ADAM (lr=0.001), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# content/style weight 10^(-4)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 20000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(20000,2,50), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# content/style weight 10^(-3)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAGRAD'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 2000
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image= model_training_and_output(epochs,steps_per_epoch,model)




file_name = '(2000,2,50), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# content/style weight 10^(-2)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAGRAD'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image= model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(200,2,50), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# content/style weight 10^(-1)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 20
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)



file_name = '(20,2,50), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
# content/style weight 100

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAGRAD'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 2
  content_weight = 200
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(2,200,50), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#  DIFFERENT TOTAL VARIATION WEIGHT (1000)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAGRAD'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 1000
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  image=model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)



file_name = '(200000,2,1000), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#  DIFFERENT TOTAL VARIATION WEIGHT (10^(-1))

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAGRAD'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 0.1
  model.set_loss_weights(style_weight,content_weight,variational_weight)
  

  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  image=model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)



file_name = '(200000,2,0.1), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#DIFFERENT STYLE LAYERS (FIRST 3)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image= model_training_and_output(epochs,steps_per_epoch,model)

file_name = '(200000,2,50), ADAM (lr=0.01), style first 3, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#DIFFERENT STYLE LAYERS (FIRST 2)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image= model_training_and_output(epochs,steps_per_epoch,model)

file_name = '(200000,2,50), ADAM (lr=0.01), style first 2, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#DIFFERENT STYLE LAYERS (FIRST ONLY)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image= model_training_and_output(epochs,steps_per_epoch,model)

file_name = '(200000,2,50), ADAM (lr=0.01), style first only, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#DIFFERENT STYLE LAYERS (LAST ONLY)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image= model_training_and_output(epochs,steps_per_epoch,model)

file_name = '(200000,2,50), ADAM (lr=0.01), style LAST only, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#DIFFERENT STYLE LAYERS (LAST 2)

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 10
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block4_conv1','block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image= model_training_and_output(epochs,steps_per_epoch,model)

file_name = '(200000,2,50), ADAM (lr=0.01), style LAST 2, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

# DIFFERENT OPTIMIZERS

In [ ]:
# RMSPROP

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'RMSPROP'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 30
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(200000,2,50), ADAM (lr=0.01), style all 5, block5_2, epoch10(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)



In [ ]:
# ADADELTA

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADADELTA'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 30
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(200000,2,50), ADAM (lr=0.01), style all 5, block5_2, epoch40(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)



In [ ]:
# ADAMAX

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAMAX'
  lr = 0.01
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 30
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 200000 
  content_weight = 2
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  model.set_style_image(tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'))
  content_path = "/content/gdrive/MyDrive/immagine_paper.jpg"
  model.set_content_image(content_path)
  model.set_training_image(content_path)

  #posso anche cambiare i layers
  content_layers = ['block5_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)



  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = '(200000,2,50), ADAM (lr=0.01), style all 5, block5_2, epoch40(x100).png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)



# WITH WHITE NOISE IMAGE

In [ ]:
if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)
 
  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.005
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 40
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)
 
  # per cambiare i pesi delle loss posso fare
  style_weight = 1e-1
  content_weight = 100
  variational_weight = 100
  model.set_loss_weights(style_weight,content_weight,variational_weight)
 
 
  # posso cambiare le immagini
  
  url = 'https://www.artble.com/imgs/c/d/1/98090/the_shipwreck_of_the_minotaur.jpg'
  
  response = requests.get(url)
 
  file = open("the_shipwreck_of_the_minotaur.jpg", "wb")
  file.write(response.content)
  file.close()
 
  model.set_style_image("the_shipwreck_of_the_minotaur.jpg")
 
 
  #c'è anche per il content image
  
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  response = requests.get(url)
  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()
 
  model.set_content_image("saverio.jpg")
  
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  
  model.util.tensor_to_image(training_image).save("noise.jpg")
  
 
  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)
  #model.set_training_image(tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'))
  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)
 
 
  image=model_training_and_output(epochs,steps_per_epoch,model)
 
 
file_name = 'paper_fig2_minotaur.png'
model.util.tensor_to_image(image).save(file_name)
 
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

In [ ]:
#different content/style weight (1000) and total wariation weight=50

if __name__ == "__main__":
  util = Utils()
  model = ModelStructure(util)
  #model_training_and_output(10,100,model)

  # cambio dei settaggi, solo per l'optimizer e i relativi parametri associati
  optimizer = 'ADAM'
  lr = 0.03
  momentum = 0.9
  BETA1 = 0.9
  BETA2 = 0.9
  initial_accumulator_value = 0.01
  epsilon = 1e-7
  rho = 0.95
  epochs = 12
  steps_per_epoch = 100
  model.set_optimizer(optimizer,lr,momentum,BETA1,BETA2,initial_accumulator_value,epsilon,rho)
  #model_training_and_output(epochs,steps_per_epoch,model)

  # per cambiare i pesi delle loss posso fare
  style_weight = 1e-1
  content_weight = 100
  variational_weight = 50
  model.set_loss_weights(style_weight,content_weight,variational_weight)


  # posso cambiare le immagini
  
  url = 'https://www.artble.com/imgs/c/d/1/98090/the_shipwreck_of_the_minotaur.jpg'
  
  response = requests.get(url)

  file = open("the_shipwreck_of_the_minotaur.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_style_image("the_shipwreck_of_the_minotaur.jpg")


  #c'è anche per il content image
  
  url = 'https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg'
  response = requests.get(url)
  file = open("Tuebingen_Neckarfront.jpg", "wb")
  file.write(response.content)
  file.close()

  model.set_content_image("Tuebingen_Neckarfront.jpg")
  
  training_image = tf.random.uniform(model.content_image.shape, minval=0, maxval=254, dtype=tf.dtypes.float32, seed=None, name=None)
  
  model.util.tensor_to_image(training_image).save("noise.jpg")
  

  training_image_path="noise.jpg"
  model.set_training_image(training_image_path)
  #model.set_training_image(tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'))
  #posso anche cambiare i layers
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']
  model.set_vgg_layers(style_layers, content_layers)


  image=model_training_and_output(epochs,steps_per_epoch,model)


file_name = 'paper_fig2_minotaur.png'
model.util.tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

### Import and configure modules

In [ ]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [ ]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
style_path = "/content/sample_data/kakashi.jpg"
content_path= "/content/gdrive/MyDrive/carlo.jpg"

Download images and choose a style image and a content image:

In [ ]:
#content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
#style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

## Visualize the input

Define a function to load an image and limit its maximum dimension to 512 pixels.

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

Create a simple function to display an image:

In [ ]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

## Fast Style Transfer using TF-Hub

This tutorial demonstrates the original style-transfer algorithm, which optimizes the image content to a particular style. Before getting into the details, let's see how the [TensorFlow Hub model](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2) does this:

In [ ]:
import tensorflow_hub as hub


hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)

## Define content and style representations

Use the intermediate layers of the model to get the *content* and *style* representations of the image. Starting from the network's input layer, the first few layer activations represent low-level features like edges and textures. As you step through the network, the final few layers represent higher-level features—object parts like *wheels* or *eyes*. In this case, you are using the VGG19 network architecture, a pretrained image classification network. These intermediate layers are necessary to define the representation of content and style from the images. For an input image, try to match the corresponding style and content target representations at these intermediate layers.


Load a [VGG19](https://keras.io/applications/#vgg19) and test run it on our image to ensure it's used correctly:

In [ ]:
x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
prediction_probabilities = vgg(x)
prediction_probabilities.shape

In [ ]:
predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
[(class_name, prob) for (number, class_name, prob) in predicted_top_5]

Now load a `VGG19` without the classification head, and list the layer names

In [ ]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

print()
for layer in vgg.layers:
  print(layer.name)

Choose intermediate layers from the network to represent the style and content of the image:


In [ ]:
content_layers = ['block5_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

#### Intermediate layers for style and content

So why do these intermediate outputs within our pretrained image classification network allow us to define style and content representations?

At a high level, in order for a network to perform image classification (which this network has been trained to do), it must understand the image. This requires taking the raw image as input pixels and building an internal representation that converts the raw image pixels into a complex understanding of the features present within the image.

This is also a reason why convolutional neural networks are able to generalize well: they’re able to capture the invariances and defining features within classes (e.g. cats vs. dogs) that are agnostic to background noise and other nuisances. Thus, somewhere between where the raw image is fed into the model and the output classification label, the model serves as a complex feature extractor. By accessing intermediate layers of the model, you're able to describe the content and style of input images.

## Build the model 

The networks in `tf.keras.applications` are designed so you can easily extract the intermediate layer values using the Keras functional API.

To define a model using the functional API, specify the inputs and outputs:

`model = Model(inputs, outputs)`

This following function builds a VGG19 model that returns a list of intermediate layer outputs:

In [ ]:
def vgg_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

And to create the model:

In [ ]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image*255)

#Look at the statistics of each layer's output
for name, output in zip(style_layers, style_outputs):
  print(name)
  print("  shape: ", output.numpy().shape)
  print("  min: ", output.numpy().min())
  print("  max: ", output.numpy().max())
  print("  mean: ", output.numpy().mean())
  print()

## Calculate style

The content of an image is represented by the values of the intermediate feature maps.

It turns out, the style of an image can be described by the means and correlations across the different feature maps. Calculate a Gram matrix that includes this information by taking the outer product of the feature vector with itself at each location, and averaging that outer product over all locations. This Gram matrix can be calculated for a particular layer as:

$$G^l_{cd} = \frac{\sum_{ij} F^l_{ijc}(x)F^l_{ijd}(x)}{IJ}$$

This can be implemented concisely using the `tf.linalg.einsum` function:

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

## Extract style and content


Build a model that returns the style and content tensors.

In [ ]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

When called on an image, this model returns the gram matrix (style) of the `style_layers` and content of the `content_layers`:

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content_image))

print('Styles:')
for name, output in sorted(results['style'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())
  print()

print("Contents:")
for name, output in sorted(results['content'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())


## Run gradient descent

With this style and content extractor, you can now implement the style transfer algorithm. Do this by calculating the mean square error for your image's output relative to each target, then take the weighted sum of these losses.

Set your style and content target values:

In [ ]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

Define a `tf.Variable` to contain the image to optimize. To make this quick, initialize it with the content image (the `tf.Variable` must be the same shape as the content image):

In [ ]:
image = tf.Variable(content_image)

Since this is a float image, define a function to keep the pixel values between 0 and 1:

In [ ]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

Create an optimizer. The paper recommends LBFGS, but `Adam` works okay, too:

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

To optimize this, use a weighted combination of the two losses to get the total loss:

In [ ]:
style_weight=10
content_weight=1e4

In [ ]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

Use `tf.GradientTape` to update the image.

In [ ]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

Now run a few steps to test:

In [ ]:
train_step(image)
train_step(image)
train_step(image)
tensor_to_image(image)

Since it's working, perform a longer optimization:

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))
  
end = time.time()
print("Total time: {:.1f}".format(end-start))

## Total variation loss

One downside to this basic implementation is that it produces a lot of high frequency artifacts. Decrease these using an explicit regularization term on the high frequency components of the image. In style transfer, this is often called the *total variation loss*:

In [ ]:
def high_pass_x_y(image):
  x_var = image[:,:,1:,:] - image[:,:,:-1,:]
  y_var = image[:,1:,:,:] - image[:,:-1,:,:]

  return x_var, y_var

In [ ]:
x_deltas, y_deltas = high_pass_x_y(content_image)

plt.figure(figsize=(14,10))
plt.subplot(2,2,1)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Original")

plt.subplot(2,2,2)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Original")

x_deltas, y_deltas = high_pass_x_y(image)

plt.subplot(2,2,3)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Styled")

plt.subplot(2,2,4)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Styled")

This shows how the high frequency components have increased.

Also, this high frequency component is basically an edge-detector. You can get similar output from the Sobel edge detector, for example:

In [ ]:
plt.figure(figsize=(14,10))

sobel = tf.image.sobel_edges(content_image)
plt.subplot(1,2,1)
imshow(clip_0_1(sobel[...,0]/4+0.5), "Horizontal Sobel-edges")
plt.subplot(1,2,2)
imshow(clip_0_1(sobel[...,1]/4+0.5), "Vertical Sobel-edges")

The regularization loss associated with this is the sum of the squares of the values:

In [ ]:
def total_variation_loss(image):
  x_deltas, y_deltas = high_pass_x_y(image)
  return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

In [ ]:
total_variation_loss(image).numpy()

That demonstrated what it does. But there's no need to implement it yourself, TensorFlow includes a standard implementation:

In [ ]:
tf.image.total_variation(image).numpy()

## Re-run the optimization

Choose a weight for the `total_variation_loss`:

In [ ]:
total_variation_weight=30

Now include it in the `train_step` function:

In [ ]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    loss += total_variation_weight*tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

Reinitialize the optimization variable:

In [ ]:
image = tf.Variable(content_image)

And run the optimization:

In [ ]:
import time
start = time.time()
epochs = 30
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))

end = time.time()
print("Total time: {:.1f}".format(end-start))

Finally, save the result:

In [ ]:
file_name = 'Carllok.png'
tensor_to_image(stylized_image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

## Learn more

This tutorial demonstrates the original style-transfer algorithm. For a simple application of style transfer check out this [tutorial](https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization) to learn more about how to use the arbitrary image style transfer model from [TensorFlow Hub](https://tfhub.dev).